In [1]:
import pickle
import pandas as pd
import sys

from tqdm.auto import tqdm

sys.path.append("../..")
pd.set_option('display.max_columns', 50)

device = "cuda"
SFREQ = 100

/home/sneddy/miniconda3/envs/eeg/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("../../artefacts/data/windows_ds_ch2_release_11.pkl", "rb") as f:
    train_raw_dataset = pickle.load(f)
train_raw_dataset.description

,subject,run,task,age,sex,p_factor,release_number,ehq_total,commercial_use,full_pheno,attention,internalizing,externalizing,restingstate,despicableme,funwithfractals,thepresent,diaryofawimpykid,contrastchangedetection_1,contrastchangedetection_2,contrastchangedetection_3,surroundsupp_1,surroundsupp_2,seqlearning6target,seqlearning8target,symbolsearch
0,NDARAB678VYW,2,contrastChangeDetection,20.1817,M,0.887,R11,100.00,Yes,Yes,0.032,0.101,-0.825,available,available,available,available,available,available,available,available,available,available,unavailable,unavailable,available
1,NDARAB678VYW,1,contrastChangeDetection,20.1817,M,0.887,R11,100.00,Yes,Yes,0.032,0.101,-0.825,available,available,available,available,available,available,available,available,available,available,unavailable,unavailable,available
2,NDARAB678VYW,3,contrastChangeDetection,20.1817,M,0.887,R11,100.00,Yes,Yes,0.032,0.101,-0.825,available,available,available,available,available,available,available,available,available,available,unavailable,unavailable,available
3,NDARAB683CYD,1,contrastChangeDetection,7.2728,M,-1.487,R11,33.35,Yes,No,0.118,-0.648,-0.450,unavailable,caution,caution,caution,caution,caution,unavailable,unavailable,caution,unavailable,unavailable,unavailable,caution
4,NDARAC296UCB,1,contrastChangeDetection,22.0002,M,0.015,R11,80.04,Yes,No,-1.159,0.098,-0.668,caution,caution,unavailable,unavailable,caution,caution,caution,unavailable,caution,unavailable,unavailable,unavailable,caution
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,NDARZV406UFK,1,contrastChangeDetection,12.0888,M,-0.390,R11,93.34,Yes,Yes,-0.201,1.023,0.228,caution,caution,caution,caution,caution,caution,caution,caution,caution,caution,unavailable,unavailable,caution
754,NDARZV406UFK,2,contrastChangeDetection,12.0888,M,-0.390,R11,93.34,Yes,Yes,-0.201,1.023,0.228,caution,caution,caution,caution,caution,caution,caution,caution,caution,caution,unavailable,unavailable,caution
755,NDARZW623WYG,1,contrastChangeDetection,11.9490,M,0.610,R11,93.34,Yes,Yes,-0.353,0.018,-0.309,unavailable,caution,caution,caution,caution,caution,caution,caution,caution,caution,unavailable,unavailable,caution
756,NDARZW623WYG,2,contrastChangeDetection,11.9490,M,0.610,R11,93.34,Yes,Yes,-0.353,0.018,-0.309,unavailable,caution,caution,caution,caution,caution,caution,caution,caution,caution,unavailable,unavailable,caution


In [3]:
from neurosned.externalizing.feature_extractor import ExternalizingFeaturesExtractor

feats_list = []
skip_num = 20
fe = ExternalizingFeaturesExtractor(lags=(5, 10, 25, 50, 100))
for run_ds in tqdm(train_raw_dataset.datasets):
    meta = {
        'externalizing': run_ds.description["externalizing"],
        'run_id': run_ds.description['run'],
        'subject_id': run_ds.description["subject"]
    }
    run_id = run_ds.description['run']
    for idx, elem in enumerate(run_ds):
        if idx % skip_num == 0:
            eeg = elem[0][:, :200]
            features, matrixes = fe.extract_features(eeg)
            features.update(meta)
            feats_list.append(features)
features = pd.DataFrame(feats_list)
features

100%|██████████| 758/758 [03:49<00:00,  3.30it/s]


,lag5_corr_mean_abs,lag5_corr_diag_mean,lag5_corr_off_mean,lag5_corr_entropy,lag5_A_mean_abs,lag5_A_diag_mean,lag5_A_off_mean,lag5_A_fro,lag5_A_asym_mean_abs,lag5_A_sparsity@0.05,lag10_corr_mean_abs,lag10_corr_diag_mean,lag10_corr_off_mean,lag10_corr_entropy,lag10_A_mean_abs,lag10_A_diag_mean,lag10_A_off_mean,lag10_A_fro,lag10_A_asym_mean_abs,lag10_A_sparsity@0.05,lag25_corr_mean_abs,lag25_corr_diag_mean,lag25_corr_off_mean,lag25_corr_entropy,lag25_A_mean_abs,...,lag25_A_asym_mean_abs,lag25_A_sparsity@0.05,lag50_corr_mean_abs,lag50_corr_diag_mean,lag50_corr_off_mean,lag50_corr_entropy,lag50_A_mean_abs,lag50_A_diag_mean,lag50_A_off_mean,lag50_A_fro,lag50_A_asym_mean_abs,lag50_A_sparsity@0.05,lag100_corr_mean_abs,lag100_corr_diag_mean,lag100_corr_off_mean,lag100_corr_entropy,lag100_A_mean_abs,lag100_A_diag_mean,lag100_A_off_mean,lag100_A_fro,lag100_A_asym_mean_abs,lag100_A_sparsity@0.05,externalizing,run_id,subject_id
0,0.951082,0.980879,0.950849,9.702513,0.009819,0.051711,0.007397,3.619800,0.006623,0.989183,0.945373,0.971120,0.945172,9.702464,0.011044,0.036333,0.007493,3.996084,0.010726,0.979148,0.927441,0.943449,0.927316,9.702115,0.014831,...,0.020712,0.926687,0.902249,0.908868,0.902197,9.702060,0.028371,0.016668,0.005194,8.679472,0.043221,0.853795,0.924926,0.929726,0.924889,9.703304,0.033544,-0.019226,0.008290,9.962540,0.045559,0.858843,-0.825,2,NDARAB678VYW
1,0.327422,0.492386,0.033992,9.512185,0.152468,0.057616,-0.004849,26.594938,0.222127,0.281113,0.162424,0.236548,0.016790,9.458664,0.159104,0.016264,0.000081,28.548531,0.233451,0.288024,0.132618,-0.039049,-0.008226,9.338587,0.166301,...,0.240891,0.261162,0.119016,-0.005705,0.004330,9.342669,0.144692,0.002698,-0.000378,25.491650,0.209972,0.283697,0.203108,-0.150608,0.007254,9.399150,0.119727,-0.005521,-0.003297,21.672873,0.177511,0.327444,-0.825,2,NDARAB678VYW
2,0.108066,0.055448,-0.011273,9.385655,0.174908,0.048511,-0.000870,30.171751,0.251096,0.252990,0.109994,0.070511,-0.000740,9.387965,0.184420,0.011912,-0.003097,31.683516,0.264340,0.236825,0.105150,-0.077316,-0.035612,9.359575,0.201535,...,0.287095,0.204855,0.091567,-0.010679,0.005457,9.313070,0.217440,0.000746,0.001014,37.404549,0.312117,0.200288,0.138767,0.031442,0.047782,9.375692,0.203083,0.005886,0.002894,34.192940,0.289493,0.201971,-0.825,2,NDARAB678VYW
3,0.347024,0.619068,0.071318,9.475776,0.124249,0.056296,0.003655,22.093390,0.182394,0.327324,0.193959,0.325690,0.046597,9.400247,0.137647,0.015836,-0.000391,24.902124,0.202340,0.290968,0.293658,-0.169244,-0.026652,9.432089,0.095987,...,0.138593,0.393306,0.188986,-0.098878,-0.027861,9.350086,0.078070,0.005690,-0.002537,14.609133,0.117120,0.496605,0.157806,-0.156264,-0.009905,9.391450,0.066476,-0.004820,-0.004996,12.479653,0.097762,0.552731,-0.825,2,NDARAB678VYW
4,0.610808,0.718880,0.099228,9.582117,0.113912,0.048651,-0.002917,20.528034,0.165570,0.331170,0.442212,0.476169,0.076493,9.563319,0.160784,0.024103,-0.003896,29.111073,0.235642,0.261282,0.194065,-0.062057,0.027013,9.308769,0.242803,...,0.349553,0.176011,0.144114,-0.038031,0.013525,9.277909,0.255856,-0.007981,0.003204,48.036694,0.380293,0.204194,0.392749,0.257060,0.092801,9.521448,0.249158,-0.001962,0.001207,42.820286,0.364203,0.159065,-0.825,2,NDARAB678VYW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6912,0.191560,0.377069,0.028909,9.394187,0.149076,0.058960,0.000445,25.162123,0.211610,0.249504,0.175802,0.324459,0.040964,9.404029,0.163181,0.019962,-0.003137,27.953690,0.232667,0.236524,0.172610,0.127398,0.003355,9.419593,0.159927,...,0.228438,0.231176,0.180337,-0.053033,-0.063342,9.408302,0.189892,0.008285,0.000632,32.330307,0.273482,0.209062,0.182795,-0.161491,-0.048521,9.370559,0.213447,-0.013885,-0.001018,35.959003,0.305176,0.185385,-0.309,3,NDARZW623WYG
6913,0.196202,0.369043,0.128164,9.387654,0.157255,0.045318,0.000568,26.566793,0.223333,0.24241

In [ ]:
features.to_csv('../../artefacts/data/externalizing_features.csv', index=False)